In [7]:
'''Ablation of NCA intervals'''
# =================================
# ============ general ============
# =================================
'''Change seed inject interval and lower down motion weight loss, 
and also enable changing texture target
and also enable reflection padding'''
img_size_x=256
img_size_y=256
motion_img_size_x = 256
motion_img_size_y = 256
video_length=20

# =================================
# ===========  Models  ============
# =================================
vqae_model="ldm-vq-f8"


# =================================
# =========== NCA ============
# =================================
'''NCA step affects the results, birds need large steps to fit the motion (min 96 is fine) 
but flag needs small steps min 32'''
nca_pool_size=256
nca_step_range_min=80
nca_step_range_max=144 
nca_inject_seed_step=8 
nca_channels=16
nca_fc_dim=128
nca_filter_scale=1.0
nca_seed_mode="zeros"
nca_random_seed=42
nca_delta_T_std=0.0
half_channel = 0
condition_nca_channel = 2



# =================================
# ============= loss ==============
# =================================
texture_loss_weight=1.0
overflow_loss_weight=1.0
regularization_loss_weight=0.01


# declare -a augments=("Cr" "Ji")
num_augs=8
texture_model = 'vgg' # ldm-vq-f16
texture_loss_type = 'OT' # OT Gram

# =================================
# ============= Gaussian Pyramid, Not used in current code ==============
# =================================
kernel_size = 3
max_level = 5
'''increase overflow loss and remove hier&sort to see effect (low quality image)'''
'''add hier first and then add sort to see if can improve quality '''
# =================================
# ========  Optimization  =========
# =================================
iterations=5000
save_every=501
motion_loss_iteration = 0#1000
batch_size=4
lr=0.001
lr_decay_step=0
optimizer="Adam"
device="cuda:0" 
# "./image/video/flag_2.gif","./image/video/fireplace_2.gif","./image/video/sea_2.gif", "./image/video/fireplace_1.gif","./image/video/flag.gif","./image/video/candy_1.gif","./image/video/smoke.gif","./image/video/birds.gif","./image/video/snake_4.gif","./image/video/cranberries.gif"
#,"./image/video/birds.gif","./image/video/cranberries.gif","./image/video/flag.gif","./image/video/candy_1.gif","./image/video/escalator.gif","./image/video/flames.gif","./image/video/fish.gif"
output_dir="out/DeepNCA_Texture_VideoMotionClean/"
dataset_path = "/scratch/students/2022-fall-sp-yitao/Dataset/DynTex/dynamic_textures/"
import os
all_folders = os.listdir(dataset_path)
paths = []
selected = ['flag_2', 'candy_1', 'birds', 'flames']
for texture_name in all_folders:
    if(texture_name in selected):
        paths.append(os.path.join(dataset_path, texture_name, f'{texture_name}.gif'))
# for texture_name in all_folders:
#     if(texture_name not in selected):
#         paths.append(os.path.join(dataset_path, texture_name, f'{texture_name}.gif'))
texture_num = len(paths)
for vqae_model in ["no-vqae"]: # "ldm-vq-f4-noattn","ldm-vq-f8","ldm-vq-f16","no-vqae"
  for style_path in paths:#,"./image/video/flag_2.gif","./image/video/sea_2.gif","./image/video/fireplace_2.gif","./image/video/escalator.gif","./image/video/flames.gif","./image/video/fish.gif"]: # 
    # for static_style_path in ["./image/video/fireplace_2.gif"]: #"./image/style/escalator.png","./image/style/smoke.png","./image/style/river.png","./image/style/ocean.png","./image/style/fish.png","./image/style/birds.png"
    for motion_texture_loss_weight in [1.0]: # 0.0 means disable motion loss
        for motion_model_name in ["two_stream_dynamic"]: # "raft-things"
            for condition_nca_channel in [0]:
                for nca_circular_pad in [1]:
                    for motion_texture_loss_type in ["MotionOT"]:
                        for motion_nca_interval in [64]:
                            for anneal in [0]:
                                if("ldm" in vqae_model):
                                    batch_size = 2
                                else:
                                    batch_size = 3
                                    regularization_loss_weight = 0.0
                                nca_perception_scales = [0]
                                if(img_size_x == 256):
                                    nca_perception_scales = [0, 1]
                                img_name = style_path.split('/')[-1].split('.')[0]
                                texture_name = img_name
                                img_name += f"-{style_path.split('/')[-1].split('.')[0]}"
                                empty_str = "x"
                                nca_perception_scales_str = f'per{empty_str.join([str(x) for x in nca_perception_scales])}'
                                cur_output_dir = f'./{output_dir}no_vqae/{img_name}/{motion_texture_loss_type}-{motion_texture_loss_weight}-{nca_step_range_min}-{nca_step_range_max}-{motion_nca_interval}-{img_size_x}-{motion_img_size_x}-nq-{nca_channels}-{nca_fc_dim}-c{condition_nca_channel}-pe-na-cp{"-anneal" if anneal==1 else ""}-1-{f"{nca_perception_scales_str}"}/'
                                final_video_path = os.path.join(cur_output_dir, "video_large.mp4")
                                # if(os.path.exists(final_video_path) == False):
                                print(f"Run!{img_name}")
                                !python fit_deep_nca_video_warmup.py --video --video_length $video_length \
                                  --output_dir $output_dir --img_size $img_size_x $img_size_y --motion_img_size $motion_img_size_x $motion_img_size_y\
                                  --style_path $style_path --static_style_path $style_path\
                                  --texture_loss_type $texture_loss_type \
                                  --nca_pool_size $nca_pool_size --nca_step_range $nca_step_range_min $nca_step_range_max \
                                  --nca_inject_seed_step $nca_inject_seed_step --nca_channels $nca_channels \
                                  --nca_fc_dim $nca_fc_dim \
                                  --nca_seed_mode $nca_seed_mode \
                                  --nca_pad_mode 'replicate' \
                                  --motion_nca_interval $motion_nca_interval --nca_pos_emb 'CPE'\
                                  --nca_perception_scales 0 1 --nca_warmup_iter 1000\
                                  --motion_model_name $motion_model_name \
                                  --motion_texture_loss_type $motion_texture_loss_type --motion_texture_loss_weight $motion_texture_loss_weight\
                                  --overflow_loss_weight $overflow_loss_weight --regularization_loss_weight $regularization_loss_weight\
                                  --iterations $iterations --save_every $save_every --batch_size $batch_size --lr $lr \
                                  --lr_decay_step 1000 2000 --DEVICE $device 
# #--hierarchy_motion_loss

Run!flag_2-flag_2
Image Size For MotionTextureLoss:  [256, 256]
Successfully Loaded two_stream_dynamic model
Preparing Style Video
Select 10 frame
Motion From flag_2, Texture From flag_2
Create NCA model
Current Multi Perception:  [0, 1]
NCA Pool shape:  torch.Size([256, 16, 256, 256])
NCA param count: 10640
  0%|                                                                                                                                                                                                                                  | 0/4000 [00:00<?, ?it/s]Set motion texture loss weight to 5.0
Warm Up Ends. Re-Initialize NCA
Set motion texture loss weight to 6.18
Current Multi Perception:  [0, 1]
 53%|████████████████████████████████████████████████████████████████████████████▊                                                                   | 2132/4000 [33:53<26:56,  1.16it/s, overflow=0.000353, texture=1.89, motion_texture=1.23, lr=9e-5, fidx=0]

Exception ignored in: <function PtyProcess.__del__ at 0x7f6e4ce011f0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/ptyprocess/ptyprocess.py", line 370, in __del__
    def __del__(self):
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
'''Ablation of NCA intervals'''
# =================================
# ============ general ============
# =================================
'''Change seed inject interval and lower down motion weight loss, 
and also enable changing texture target
and also enable reflection padding'''
img_size_x=256
img_size_y=256
motion_img_size_x = 256
motion_img_size_y = 256
video_length=20

# =================================
# ===========  Models  ============
# =================================
vqae_model="ldm-vq-f8"


# =================================
# =========== NCA ============
# =================================
'''NCA step affects the results, birds need large steps to fit the motion (min 96 is fine) 
but flag needs small steps min 32'''
nca_pool_size=256
nca_step_range_min=80
nca_step_range_max=144 
nca_inject_seed_step=8 
nca_channels=12
nca_fc_dim=96
nca_filter_scale=1.0
nca_seed_mode="zeros"
nca_random_seed=42
nca_delta_T_std=0.0
half_channel = 0
condition_nca_channel = 2



# =================================
# ============= loss ==============
# =================================
texture_loss_weight=1.0
overflow_loss_weight=1.0
regularization_loss_weight=0.01


# declare -a augments=("Cr" "Ji")
num_augs=8
texture_model = 'vgg' # ldm-vq-f16
texture_loss_type = 'OT' # OT Gram

# =================================
# ============= Gaussian Pyramid, Not used in current code ==============
# =================================
kernel_size = 3
max_level = 5
'''increase overflow loss and remove hier&sort to see effect (low quality image)'''
'''add hier first and then add sort to see if can improve quality '''
# =================================
# ========  Optimization  =========
# =================================
iterations=5000
save_every=501
motion_loss_iteration = 0#1000
batch_size=4
lr=0.001
lr_decay_step=0
optimizer="Adam"
device="cuda:0" 
# "./image/video/flag_2.gif","./image/video/fireplace_2.gif","./image/video/sea_2.gif", "./image/video/fireplace_1.gif","./image/video/flag.gif","./image/video/candy_1.gif","./image/video/smoke.gif","./image/video/birds.gif","./image/video/snake_4.gif","./image/video/cranberries.gif"
#,"./image/video/birds.gif","./image/video/cranberries.gif","./image/video/flag.gif","./image/video/candy_1.gif","./image/video/escalator.gif","./image/video/flames.gif","./image/video/fish.gif"
output_dir="out/DeepNCA_Texture_VideoMotionClean/"
dataset_path = "/scratch/students/2022-fall-sp-yitao/Dataset/DynTex/dynamic_textures/"
import os
all_folders = os.listdir(dataset_path)
paths = []
selected = ['flag_2', 'candy_1', 'birds', 'flames']
for texture_name in all_folders:
    if(texture_name in selected):
        paths.append(os.path.join(dataset_path, texture_name, f'{texture_name}.gif'))
# for texture_name in all_folders:
#     if(texture_name not in selected):
#         paths.append(os.path.join(dataset_path, texture_name, f'{texture_name}.gif'))
texture_num = len(paths)
for vqae_model in ["no-vqae"]: # "ldm-vq-f4-noattn","ldm-vq-f8","ldm-vq-f16","no-vqae"
  for style_path in paths:#,"./image/video/flag_2.gif","./image/video/sea_2.gif","./image/video/fireplace_2.gif","./image/video/escalator.gif","./image/video/flames.gif","./image/video/fish.gif"]: # 
    # for static_style_path in ["./image/video/fireplace_2.gif"]: #"./image/style/escalator.png","./image/style/smoke.png","./image/style/river.png","./image/style/ocean.png","./image/style/fish.png","./image/style/birds.png"
    for motion_texture_loss_weight in [1.0]: # 0.0 means disable motion loss
        for motion_model_name in ["two_stream_dynamic"]: # "raft-things"
            for condition_nca_channel in [0]:
                for nca_circular_pad in [1]:
                    for motion_texture_loss_type in ["MotionOT"]:
                        for motion_nca_interval in [64]:
                            for anneal in [0]:
                                if("ldm" in vqae_model):
                                    batch_size = 2
                                else:
                                    batch_size = 3
                                    regularization_loss_weight = 0.0
                                nca_perception_scales = [0]
                                if(img_size_x == 256):
                                    nca_perception_scales = [0, 1]
                                img_name = style_path.split('/')[-1].split('.')[0]
                                texture_name = img_name
                                img_name += f"-{style_path.split('/')[-1].split('.')[0]}"
                                empty_str = "x"
                                nca_perception_scales_str = f'per{empty_str.join([str(x) for x in nca_perception_scales])}'
                                cur_output_dir = f'./{output_dir}no_vqae/{img_name}/{motion_texture_loss_type}-{motion_texture_loss_weight}-{nca_step_range_min}-{nca_step_range_max}-{motion_nca_interval}-{img_size_x}-{motion_img_size_x}-nq-{nca_channels}-{nca_fc_dim}-c{condition_nca_channel}-pe-na-cp{"-anneal" if anneal==1 else ""}-1-{f"{nca_perception_scales_str}"}/'
                                final_video_path = os.path.join(cur_output_dir, "video_large.mp4")
                                # if(os.path.exists(final_video_path) == False):
                                print(f"Run!{img_name}")
                                !python fit_deep_nca_video_warmup.py --video --video_length $video_length \
                                  --output_dir $output_dir --img_size $img_size_x $img_size_y --motion_img_size $motion_img_size_x $motion_img_size_y\
                                  --style_path $style_path --static_style_path $style_path\
                                  --texture_loss_type $texture_loss_type \
                                  --nca_pool_size $nca_pool_size --nca_step_range $nca_step_range_min $nca_step_range_max \
                                  --nca_inject_seed_step $nca_inject_seed_step --nca_channels $nca_channels \
                                  --nca_fc_dim $nca_fc_dim \
                                  --nca_seed_mode $nca_seed_mode \
                                  --nca_pad_mode 'replicate' \
                                  --motion_nca_interval $motion_nca_interval --nca_pos_emb 'CPE'\
                                  --nca_perception_scales 0 1 --nca_warmup_iter 1000\
                                  --motion_model_name $motion_model_name \
                                  --motion_texture_loss_type $motion_texture_loss_type --motion_texture_loss_weight $motion_texture_loss_weight\
                                  --overflow_loss_weight $overflow_loss_weight --regularization_loss_weight $regularization_loss_weight\
                                  --iterations $iterations --save_every $save_every --batch_size $batch_size --lr $lr \
                                  --lr_decay_step 1000 2000 --DEVICE $device 
# #--hierarchy_motion_loss

In [ ]:
'''Ablation of NCA intervals'''
# =================================
# ============ general ============
# =================================
'''Change seed inject interval and lower down motion weight loss, 
and also enable changing texture target
and also enable reflection padding'''
img_size_x=128
img_size_y=128
motion_img_size_x = 128
motion_img_size_y = 128
video_length=20

# =================================
# ===========  Models  ============
# =================================
vqae_model="ldm-vq-f8"


# =================================
# =========== NCA ============
# =================================
'''NCA step affects the results, birds need large steps to fit the motion (min 96 is fine) 
but flag needs small steps min 32'''
nca_pool_size=256
nca_step_range_min=80
nca_step_range_max=144 
nca_inject_seed_step=8 
nca_channels=16
nca_fc_dim=128
nca_filter_scale=1.0
nca_seed_mode="zeros"
nca_random_seed=42
nca_delta_T_std=0.0
half_channel = 0
condition_nca_channel = 2



# =================================
# ============= loss ==============
# =================================
texture_loss_weight=1.0
overflow_loss_weight=1.0
regularization_loss_weight=0.01


# declare -a augments=("Cr" "Ji")
num_augs=8
texture_model = 'vgg' # ldm-vq-f16
texture_loss_type = 'OT' # OT Gram

# =================================
# ============= Gaussian Pyramid, Not used in current code ==============
# =================================
kernel_size = 3
max_level = 5
'''increase overflow loss and remove hier&sort to see effect (low quality image)'''
'''add hier first and then add sort to see if can improve quality '''
# =================================
# ========  Optimization  =========
# =================================
iterations=5000
save_every=501
motion_loss_iteration = 0#1000
batch_size=4
lr=0.001
lr_decay_step=0
optimizer="Adam"
device="cuda:0" 
# "./image/video/flag_2.gif","./image/video/fireplace_2.gif","./image/video/sea_2.gif", "./image/video/fireplace_1.gif","./image/video/flag.gif","./image/video/candy_1.gif","./image/video/smoke.gif","./image/video/birds.gif","./image/video/snake_4.gif","./image/video/cranberries.gif"
#,"./image/video/birds.gif","./image/video/cranberries.gif","./image/video/flag.gif","./image/video/candy_1.gif","./image/video/escalator.gif","./image/video/flames.gif","./image/video/fish.gif"
output_dir="out/DeepNCA_Texture_VideoMotionClean/"
dataset_path = "/scratch/students/2022-fall-sp-yitao/Dataset/DynTex/dynamic_textures/"
import os
all_folders = os.listdir(dataset_path)
paths = []
selected = ['flag_2', 'candy_1', 'birds', 'flames']
for texture_name in all_folders:
    if(texture_name in selected):
        paths.append(os.path.join(dataset_path, texture_name, f'{texture_name}.gif'))
# for texture_name in all_folders:
#     if(texture_name not in selected):
#         paths.append(os.path.join(dataset_path, texture_name, f'{texture_name}.gif'))
texture_num = len(paths)
for vqae_model in ["no-vqae"]: # "ldm-vq-f4-noattn","ldm-vq-f8","ldm-vq-f16","no-vqae"
  for style_path in paths:#,"./image/video/flag_2.gif","./image/video/sea_2.gif","./image/video/fireplace_2.gif","./image/video/escalator.gif","./image/video/flames.gif","./image/video/fish.gif"]: # 
    # for static_style_path in ["./image/video/fireplace_2.gif"]: #"./image/style/escalator.png","./image/style/smoke.png","./image/style/river.png","./image/style/ocean.png","./image/style/fish.png","./image/style/birds.png"
    for motion_texture_loss_weight in [1.0]: # 0.0 means disable motion loss
        for motion_model_name in ["two_stream_dynamic"]: # "raft-things"
            for condition_nca_channel in [0]:
                for nca_circular_pad in [1]:
                    for motion_texture_loss_type in ["MotionOT"]:
                        for motion_nca_interval in [64]:
                            for anneal in [0]:
                                if("ldm" in vqae_model):
                                    batch_size = 2
                                else:
                                    batch_size = 4
                                    regularization_loss_weight = 0.0
                                nca_perception_scales = [0]
                                if(img_size_x == 256):
                                    nca_perception_scales = [0, 1]
                                img_name = style_path.split('/')[-1].split('.')[0]
                                texture_name = img_name
                                img_name += f"-{style_path.split('/')[-1].split('.')[0]}"
                                empty_str = "x"
                                nca_perception_scales_str = f'per{empty_str.join([str(x) for x in nca_perception_scales])}'
                                cur_output_dir = f'./{output_dir}no_vqae/{img_name}/{motion_texture_loss_type}-{motion_texture_loss_weight}-{nca_step_range_min}-{nca_step_range_max}-{motion_nca_interval}-{img_size_x}-{motion_img_size_x}-nq-{nca_channels}-{nca_fc_dim}-c{condition_nca_channel}-pe-na-cp{"-anneal" if anneal==1 else ""}-1-{f"{nca_perception_scales_str}"}/'
                                final_video_path = os.path.join(cur_output_dir, "video_large.mp4")
                                # if(os.path.exists(final_video_path) == False):
                                print(f"Run!{img_name}")
                                !python fit_deep_nca_video_warmup.py --video --video_length $video_length \
                                  --output_dir $output_dir --img_size $img_size_x $img_size_y --motion_img_size $motion_img_size_x $motion_img_size_y\
                                  --style_path $style_path --static_style_path $style_path\
                                  --texture_loss_type $texture_loss_type \
                                  --nca_pool_size $nca_pool_size --nca_step_range $nca_step_range_min $nca_step_range_max \
                                  --nca_inject_seed_step $nca_inject_seed_step --nca_channels $nca_channels \
                                  --nca_fc_dim $nca_fc_dim \
                                  --nca_seed_mode $nca_seed_mode \
                                  --nca_pad_mode 'replicate' \
                                  --motion_nca_interval $motion_nca_interval --nca_pos_emb 'CPE'\
                                  --nca_perception_scales 0 --nca_warmup_iter 1000\
                                  --motion_model_name $motion_model_name \
                                  --motion_texture_loss_type $motion_texture_loss_type --motion_texture_loss_weight $motion_texture_loss_weight\
                                  --overflow_loss_weight $overflow_loss_weight --regularization_loss_weight $regularization_loss_weight\
                                  --iterations $iterations --save_every $save_every --batch_size $batch_size --lr $lr \
                                  --lr_decay_step 1000 2000 --DEVICE $device 
# #--hierarchy_motion_loss